In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import standard packages
from __future__ import division
from numpy import *
import numpy as np
from scipy.sparse import coo_matrix
from matplotlib import colors
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import time
import os
import gc
import warnings


In [ ]:
# install necessary packages
!pip install tf_fourier_features
!pip install geomdl

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# path of current folder
folder_path = '/content/drive/MyDrive/DEM_TO/Heat_100_50/'
# add the path to python's search list
sys.path.append(folder_path)
sys.path.append(folder_path+'UtilityFunctions/')

In [ ]:
# import self-defined modules
import utils as ut
import surrogate_model_primal as sm_pr
from mma import mmasub

In [ ]:
# use gpu for tensorflow
if tf.config.list_physical_devices('GPU'):
  physical_devices = tf.config.list_physical_devices('GPU')
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
  print('Using GPU')
else:
  print('Using CPU')

Using GPU


In [ ]:
ut.makepath(folder_path+'TOdesign')
ut.makepath(folder_path+'surrogate_result')
ut.makepath(folder_path+'model_pr')

'/content/drive/MyDrive/DEM_TO/Heat_100_50/surrogate_result/'

In [ ]:
class TO2D_heat():
    # Initialization of the class with default parameters
    def __init__(self,Lx=100,Ly=50,nelx=100,nely=50,volfrac=0.5,rmin=3,maxiter=1000,filter=0,x_init=0.5):
        # Class parameters
        self.Lx = Lx
        self.Ly = Ly
        self.nelx = nelx
        self.nely = nely
        self.volfrac = volfrac
        self.rmin = rmin
        self.maxiter = maxiter
        self.ft = filter
        self.x_init = x_init* np.ones(self.nely*self.nelx,dtype=float)

        # Initialization message
        print("2D TO for heat sink")
        print("Number of nodes: " + str(self.nelx) + " x " + str(self.nely))
        print("Volume fraction: " + str(self.volfrac) + ", rmin: " + str(self.rmin))
        print("Filter method: " + ["Density based","Density based + Heaviside filter"][self.ft])

    # Main topology optimization routine
    def optimize(self):

        # initialize design variable and MMA paramters
        x = self.x_init.copy()
        nn = len(x)
        mm = 1
        [geps,xval,move,xmin,xmax,fmax,mma_a,mma_c] = self.mmaInit(mm, nn, self.volfrac)
        xlow = xmin
        xupp = xmax
        xold1 = xval
        xold2 = xval

        # initialize the physical density vector
        Hbeta = 1
        if self.ft==0:
            xPhys=x.copy()
        elif self.ft==1:
            xTilde=x.copy()
            xPhys=1-np.exp(-Hbeta*xTilde)+xTilde*np.exp(-Hbeta)

        # ut.DensityFilter_init gives the H and Hs for density filtering
        H,Hs = ut.DensityFilter_init(self.rmin,self.nelx,self.nely)

        # set the material properties
        k0 = 1
        kmin = 1e-3
        p = 3
        material = {'volfrac':self.volfrac, 'k': k0, 'kmin': kmin, 'p':p, 's':0.001*(100)**2/117.65}

        # obtain the gaussian quadrature points
        Xint, Wint, _, _, eleidx = ut.getquadpts(L=self.Lx,H=self.Ly,Nelmufem=self.nelx,Nelmvfem=self.nely,numElemU=self.nelx,numElemV=self.nely,numGauss=2,bdcode=1)
        # obtain the TO mesh nodes
        input_dof,evenidx,oddidx = ut.generatedofcoord(nelx=self.nelx,nely=self.nely,L=self.Lx,H=self.Ly)
        Wint = np.squeeze(Wint)
        # Nsample is the number of randomly sampled traning points (For Monte-Carlo integration)
        Nsample = 40000

        mesh = {'nelx':self.nelx,'nely':self.nely,'L':self.Lx/100,'H':self.Ly/100,\
        'eleidx':eleidx,'Xint':Xint/100,'input_dof':input_dof/100,'evenidx':evenidx,'oddidx':oddidx,'Nsample':Nsample}

        # nueral network model paramters
        lr = 1e-3
        Nepoch = 10
        batch_size = 2000
        neuron = 86
        rff_dev = 1.32
        N_Layers = 4
        act_func = 'swish'
        N_input_rho = self.nelx*self.nely
        N_input_coord = 2
        trainParams={'lr':lr,'Nepoch':Nepoch,'Nbatch':batch_size,'Nneuron':neuron,'rff_dev':rff_dev,'Nlayers':N_Layers,'activation':act_func,'N_input_rho':N_input_rho,'N_input_coord':N_input_coord}

        # initialize the surrogate model for primal equation
        self.SModel_pr = sm_pr.SurrogateModel_primal()
        self.SModel_pr.updatemesh(mesh)
        self.SModel_pr.updatematerial(material)
        self.SModel_pr.updatetrainParams(trainParams)

        # set the boundary condition training points
        X_DBC = np.zeros((5000,2))
        X_DBC[:,0] = 0.
        X_DBC[:,1] = np.linspace(0,self.Ly/10,5000)

        # Set loop counter and initialize gradient vectors
        loop=0
        loopbeta=0
        iter=0
        change=1
        dv = np.ones(self.nely*self.nelx)
        dc = np.ones(self.nely*self.nelx)
        start_time=time.time()
        # main topology optimization loop
        while change>-1 and loop<=self.maxiter:  # loop until maxiter is reached
            loop=loop+1
            loopbeta=loopbeta+1
            iter += 1

            rho_int = xPhys[eleidx] # rho_int is the value of density at gaussian quadrature points
            data = {'rho_quad':rho_int,'DBC':X_DBC/100}

            if loop==1:
              # if TO iteration=1, train longer, Nepoch=1000
              trainParams={'lr':lr,'Nepoch':1000,'Nbatch':batch_size,'Nneuron':neuron,'rff_dev':rff_dev,'Nlayers':N_Layers,'activation':act_func,'N_input_rho':N_input_rho,'N_input_coord':N_input_coord}
            else:
              # other wise Nepoch=10
              trainParams={'lr':lr,'Nepoch':Nepoch,'Nbatch':batch_size,'Nneuron':neuron,'rff_dev':rff_dev,'Nlayers':N_Layers,'activation':act_func,'N_input_rho':N_input_rho,'N_input_coord':N_input_coord}
            # update the train parameters to primal surrogate model
            self.SModel_pr.updatetrainParams(trainParams)
            self.SModel_pr.updatetrainset(data)

            if loop==1:
                # if TO iteration=1, initialize the neural network model
                self.SModel_pr.initialize_model()


            # train the neural network model
            self.SModel_pr.trainmodel(verbose=True)
            print('Train ends after %d epochs'%self.SModel_pr.Endepoch)

            # save the model
            self.SModel_pr.Model.save(folder_path+'model_pr/model_pr.h5')

            # pass TO mesh nodes' coordinates to neural network, get the temperature
            t = self.SModel_pr.Model.predict(input_dof/100)
            t = t*117.65

            # plot the current design and solution
            ut.plotdesign(folder_path+'surrogate_result/','current_design',xPhys,nelx=self.nelx,nely=self.nely)
            ut.plotresult_contour(folder_path+'surrogate_result/','solution_T',input_dof,t,L=self.Lx,H=self.Ly)

            # calculate the volume and volume constraint sensitivity
            volume = np.sum(xPhys)/(self.nelx*self.nely)
            dv = np.ones(self.nely*self.nelx)/self.nelx/self.nely

            # calculate the spatial gradients by AD
            T_dx,T_dy = self.SModel_pr.nabla_T(ut.preprocess(Xint[:,0]/100),ut.preprocess(Xint[:,1]/100))
            F_GQ = (T_dx*(117.65/self.Lx))**2+(T_dy*(117.65/self.Lx))**2
            #calculate cost
            FW_c = np.squeeze(F_GQ)*Wint*(kmin+(k0-kmin)*rho_int**3)
            cost = np.sum(FW_c)
            #calculate sensitivity
            FW_dc = np.squeeze(F_GQ)*Wint
            FW_dc = np.reshape(FW_dc,(-1,4))
            FW_dc = np.sum(FW_dc,axis=1)
            dc[:][eleidx[0::4]] = FW_dc
            dc = -1/2*(3*(xPhys**(3-1))*(k0-kmin))*dc

            if iter==1:
                cost0 = cost
            if self.ft==0:
                # if use density filter, the sensitivity...
                dc[:] = np.asarray(H*(dc[np.newaxis].T/Hs))[:,0]
                dv[:] = np.asarray(H*(dv[np.newaxis].T/Hs))[:,0]
            elif self.ft==1:
                # if use density+heaviside filter, the sensitivity...
                dx = Hbeta*np.exp(-Hbeta*xTilde)+np.exp(-Hbeta)
                dc[:] = np.asarray(H*((dc*dx)[np.newaxis].T/Hs))[:,0]
                dv[:] = np.asarray(H*((dv*dx)[np.newaxis].T/Hs))[:,0]

            xold2 = xold1.copy()
            xold1 = x.copy()
            g = volume/self.volfrac
            dg = dv/self.volfrac

            # mma gives design update
            xmin = maximum(zeros([nn,]), xval-move)
            xmax = minimum(ones([nn,]), xval+move)
            [iyfree,xmma,xlow,xupp,alfa,beta,mma_b,mma_y,mma_z,ulam,mma_p,mma_q,p0,q0,uu,gradf,dsrch,hessf] = \
                mmasub(iter,mm,nn,geps,xval,
                    xmin, xmax, xold1,xold2,xlow,xupp,
                    mma_a,mma_c,cost/cost0,g,fmax,dc/cost0, dg)
            xold2 = xold1
            xold1 = xval
            xval = xmma
            x[:] = xmma

            # filter the updated design
            if self.ft==0:	xPhys[:]=np.asarray(H*x[np.newaxis].T/Hs)[:,0]
            elif self.ft==1:
                xTilde[:] = np.asarray(H*x[np.newaxis].T/Hs)[:,0]
                xPhys[:] = 1-np.exp(-Hbeta*xTilde)+xTilde*np.exp(-Hbeta)
            change=np.abs(xmma-xold1).max()

            # if use heaviside filter, beta need to increase
            if self.ft==1 and Hbeta<512 and loopbeta >=50:# or change<=0.01):
                Hbeta = 2*Hbeta
                loopbeta = 0
                change = 1
                print('Parameter beta increased to %d.\n'%Hbeta)

            if loop>1:
                del fig, ax

            # save the TO design to folder
            fig,ax = plt.subplots()
            im = ax.imshow(xPhys.reshape((self.nelx,self.nely)).T, cmap='gray_r',\
            interpolation='none',norm=colors.Normalize(vmin=0,vmax=1))
            fig.show()
            fig.canvas.draw()
            if loop%1==0:
                plt.savefig(folder_path+'TOdesign/design%d.png'%(loop))
                plt.close('all')
                np.savetxt(folder_path+'TOdesign/design%d.txt'%(loop),xPhys)

            # print the outcome of this TO iteration
            print("it.: {0} , obj.: {1:.3f} Vol.: {2:.3f}, ch.: {3:.3f}".format(\
                        loop,cost,volume,change))

            gc.collect()

        end_time=time.time()-start_time
        print('total time is: ', end_time)
        plt.show()

    def mmaInit(self, m, n, volfrac): # function to initilize mma
        move = 0.5 # move limit
        geps = 1e-9  # 1e-7

        xmin = zeros([n,]) # 0.001*ones([3,])
        xmax= ones([n,])
        # xval = volfrac*ones([n,])
        xval = ones([n,])*self.x_init

        # vol/volfrac < 1 where RHS is fmax
        fmax = 1.*ones([m,])

        mma_a = 0.*ones([m,])
        mma_c = 1000.*ones([m,])

        return geps,xval,move, xmin,xmax,fmax,mma_a,mma_c


    def __call__(self):
        self.optimize()



In [ ]:
# initialize the clasee to2d and then conduct TO
to2d= TO2D_heat(Lx=100,Ly=50,nelx=100,nely=50,filter=0,rmin=4.2,volfrac=0.5,x_init=0.5,maxiter=500)
to2d.optimize()

2D TO for heat sink
Number of nodes: 100 x 50
Volume fraction: 0.5, rmin: 4.2
Filter method: Density based
Epoch 50:
lr: 1.00e-03
Data=5.4876e-04,               PDE=-1.0143e-02,               total_loss=-9.5946e-03
Epoch 100:
lr: 1.00e-03
Data=6.9402e-04,               PDE=-1.2730e-02,               total_loss=-1.2036e-02
Epoch 150:
lr: 1.00e-03
Data=6.0022e-04,               PDE=-1.3907e-02,               total_loss=-1.3306e-02
Epoch 200:
lr: 1.00e-03
Data=6.1251e-04,               PDE=-1.3490e-02,               total_loss=-1.2877e-02
Epoch 250:
lr: 1.00e-03
Data=7.1226e-04,               PDE=-1.3115e-02,               total_loss=-1.2403e-02
Epoch 300:
lr: 1.00e-03
Data=6.5106e-04,               PDE=-1.3925e-02,               total_loss=-1.3274e-02
Epoch 350:
lr: 1.00e-03
Data=6.9482e-04,               PDE=-1.3543e-02,               total_loss=-1.2848e-02
Epoch 400:
lr: 1.00e-03
Data=6.5048e-04,               PDE=-1.4278e-02,               total_loss=-1.3628e-02
Epoch 450:
lr: 1.00e-0